## Stock Price Prediction with RNN-LSTM Models

This guide outlines the process of building an RNN model with LSTM cells to predict stock prices. The model's predictive performance can be enhanced by incorporating exploratory data analysis techniques, such as leveraging multiple tickers that are correlated with the target ticker and applying advanced statistical preprocessing methods.

### Brief Introduction

The stock market is influenced by various factors, including macroeconomic conditions, geopolitical events, and the performance of individual companies. As the market becomes increasingly complex, algorithmic trading has emerged as a popular and effective method for analyzing and predicting stock movements. Also known as automated or black-box trading, algorithmic trading employs advanced techniques to make informed decisions in these challenging market environments.

This approach is gaining popularity since it allows traders to process and analyze vast amounts of publicly available data—far beyond the capacity of manual methods—enabling faster and more accurate decision-making. Notably, modern algorithmic trading systems excel at analyzing large-scale market data in real time, identifying opportunities, and executing trades based on predefined rules. These systems consider multiple factors, such as market trends, breaking news, and technical indicators, to enhance their predictive power and efficiency.

In recent years, machine learning (ML) techniques have become increasingly prominent in the financial industry, particularly in the stock market. Among the various ML architectures, recurrent neural networks (RNNs) stand out for their effectiveness in processing sequential data. RNNs utilize self-loops in their hidden layers, enabling them to retain information from previous states and apply it to interpret the current state based on new input. This sequential structure makes RNNs particularly well-suited for handling time series data, such as historical stock prices and other financial metrics. By analyzing such data, RNNs can identify patterns and trends that help predict future stock prices.

Building on this foundation, the long short-term memory (LSTM) cell is a specialized RNN unit that enhances the network's ability to retain and utilize long-term context. Given that short-term trends in the stock market often have a more immediate impact on future prices than historical trends—though both are important—LSTMs are well-suited for analyzing stock market data. Additionally, LSTMs are capable of processing sequences with multiple input and output time steps, allowing them to account for the simultaneous influence of various factors on stock prices. This capability makes LSTMs highly effective at capturing complex relationships and generating more accurate predictions.

Overall, LSTM models, combined with time series analysis, are powerful tools for investors seeking to make informed decisions based on comprehensive stock market data analysis.

`Literature reviews can be added here, but not for sure :)`

### RNN-LSTM Model

<img src="figures/RNN.png" width="800">

Compared to a traditional neural network, RNNs have internal loop structures, allowing information to persist.

In the above figure, a part of RNN with some inputs $x_t$ and outputs $h_t$ is presented in the unrolled loop diagram, which roughly describes how the information flows in the network.

This chain-like nature reveals that RNNs are intimately related to sequential data.

In the last few years, there have been incredible success applying RNNs to a variety of problems: speech recognition, language modeling, translation, image captioning and multiple of other applications.

`[Karpathy]The Unreasonable Effectiveness of Recurrent Neural Networks: Related post can be added here!`

LSTMs is a special kind of RNN, which works far better than the standard version for many tasks.

One of the appeals of RNNs is the idea that they might be able to connect previous information to the present task, but sometimes, we only need to look at recent information to perform the present task.

But there are also cases where we need more context. It’s entirely possible for the gap between the relevant information and the point where it is needed to become very large. Unfortunately, as that gap grows, RNNs become unable to learn to connect the information.

In theory, RNNs are absolutely capable of handling such “long-term dependencies.” Sadly, in practice, RNNs don’t seem to be able to learn them.
`Hochreiter (1991) [German] and Bengio, et al. (1994): it describes that the fundamental adversities of such tasks.`

Thankfully, LSTMs don’t have this problem!

<img src="figures/LSTM.png" width="800">

LSTMs, introduced by Hochreiter and Schmidhuber in 1997, are a special type of RNN designed to learn and retain long-term dependencies. Unlike traditional RNNs, LSTMs excel at preserving information over extended periods, making long-term memory their inherent strength rather than a challenge to overcome.

Structurally, LSTMs resemble standard RNNs with a chain-like architecture, but their repeating module is uniquely designed. Instead of a single neural network layer, the module contains four interconnected layers that interact in a highly specialized manner.

In the accompanying diagram, each line represents an entire vector, carrying outputs from one node to the inputs of others. Circles indicate pointwise operations, such as vector addition, while the light blue boxes represent neural network layers. When lines merge, they signify concatenation, and when they fork, they indicate that the content is copied and distributed to multiple modules.

LSTMs have a unique feature called the cell state, represented by the horizontal line running through the top of the diagram.

LSTMs can add or remove information from the cell state, a process meticulously managed by structures known as gates. Gates serve as pathways that control the flow of information. Each gate is composed of a sigmoid neural network layer and a pointwise multiplication operation.

The sigmoid layer outputs values between 0 and 1, determining the extent to which each component of the information should pass through. This mechanism allows the LSTM to carefully regulate the cell state.

An LSTM employs three types of gates to protect and control the cell state. The first step involves deciding which information to discard from the cell state. This decision is made by a sigmoid layer called the forget gate layer.

$f_t = \sigma(W_f \cdot [h_{t-1}, x_t] + b_f)$

The next step is to decide what new information we’re going to store in the cell state.

This has two parts.

First, a sigmoid layer called the “input gate layer” decides which values we’ll update.

$i_t = \sigma(W_i \cdot [h_{t-1}, x_t] + b_i)$

Next, a tanh layer creates a vector of new candidate values, $\tilde{C}_t$, that could be added to the state.

$\tilde{C}_t = \tanh(W_C \cdot [h_{t-1}, x_t] + b_C)$

In the next step, we’ll combine these two to create an update to the state.


The next step in the LSTM process is determining what new information to store in the cell state. This involves two key components:

A sigmoid layer: This known as the input gate layer, decides which values in the cell state should be updated. This is mathematically represented as:

$i_t = \sigma(W_i \cdot [h_{t-1}, x_t] + b_i)$

A tanh layer generates a vector of potential new values $\tilde{C}_t$, that can be added to the cell state. This step is expressed as:

$\tilde{C}_t = \tanh(W_C \cdot [h_{t-1}, x_t] + b_C)$

In the subsequent step, these two components are combined to update the cell state.

The old cell state, $C_{t − 1}$, is now updated to form the new cell state, $C_t$.

With the decisions from the previous steps in place, the LSTM executes the update process. First, the old cell state is multiplied by $f_t$, effectively "forgetting" the information deemed unnecessary. Then, the new candidate values, scaled by the update decisions, are added:
$C_t = f_t * C_{t - 1} + i_t \ast \tilde{C}_t$.

Here, $f_t * C_{t - 1}$ represents the retained information, while $i_t \ast \tilde{C}_t$ introduces the new information to the state.




The final step in the LSTM process is determining what to output. The output is a filtered version of the cell state, influenced by the decisions made so far.

Output Gate: First, a sigmoid layer determines which parts of the cell state to output. This is represented as:

$o_t = \sigma(W_o \cdot [h_{t-1}, x_t] + b_o)$

Filtered Output: Next, the cell state is passed through a tanh function, which scales its values to the range [−1,1]. This output is then multiplied by the result of the sigmoid gate to filter the information:

$h_t = o_t \ast \tanh C_t$.

This final output, $h_t$, combines the current context in a form suitable for downstream tasks or the next step in the sequence.

### Model Construction

### Backtesting

### Real-time Prediction Results